In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
def preprocess_data(X, scaler=None):
    '''Standardize features by removing the mean and scaling to unit variance'''
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(X)
    return scaler.transform(X), scaler

def preprocess_labels(y, encoder=None, categorical=True):
    '''Encode labels with value between 0 and n_classes-1.'''
    if not encoder:
        encoder = LabelEncoder()
        encoder.fit(y)
    labels = encoder.transform(y).astype(np.int32)
    if categorical:
        labels = np_utils.to_categorical(labels)
    return labels, encoder

def load_data(path):
    df = pd.read_csv(path)
    data = df.values
    np.random.shuffle(data)
    X = data[:, 1:-1].astype(np.float32)
    y = data[:, -1]
    y, _ = preprocess_labels(y)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5)
    x_train, scaler = preprocess_data(x_train)
#     we use only the training data to normalize the data.
    x_test, _ = preprocess_data(x_test, scaler)
    x_val, _ = preprocess_data(x_val, scaler)
    return ((x_train, y_train),
            (x_val, y_val),
            (x_test, y_test))

In [3]:
path = "data/train.csv"
(x_train, y_train), (x_val, y_val), (x_test, y_test) = load_data(path=path)

In [4]:
dims = x_train.shape[1]
nb_classes = y_train.shape[1]
print("Number of dimensions =", dims)
print("Number of classes =", nb_classes)

Number of dimensions = 93
Number of classes = 9


In [5]:
model = Sequential()
model.add(Dense(100, input_shape=(dims,)))
# model.add(Dense(100))
# model.add(Dense(100))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy')

path_to_model = 'model/model.h5'
early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=1) 
best_model = ModelCheckpoint(path_to_model, verbose=0, save_best_only=True)

model.fit(x_train, y_train,
          validation_data = (x_val, y_val),
          nb_epoch=100,
          batch_size=128,
          verbose = True,
          callbacks=[early_stop, best_model])

model.evaluate(x_val, y_val, verbose=1)

Train on 43314 samples, validate on 9282 samples
Epoch 1/100
43314/43314 [==============================] - 1s - loss: 1.2662 - val_loss: 0.9017
Epoch 2/100
43314/43314 [==============================] - 1s - loss: 0.8409 - val_loss: 0.7987
Epoch 3/100
43314/43314 [==============================] - 2s - loss: 0.7731 - val_loss: 0.7586
Epoch 4/100
43314/43314 [==============================] - 2s - loss: 0.7400 - val_loss: 0.7359
Epoch 5/100
43314/43314 [==============================] - 1s - loss: 0.7195 - val_loss: 0.7214
Epoch 6/100
43314/43314 [==============================] - 1s - loss: 0.7054 - val_loss: 0.7103
Epoch 7/100
43314/43314 [==============================] - 1s - loss: 0.6947 - val_loss: 0.7032
Epoch 8/100
43314/43314 [==============================] - 1s - loss: 0.6867 - val_loss: 0.6968
Epoch 9/100
43314/43314 [==============================] - 1s - loss: 0.6803 - val_loss: 0.6921
Epoch 10/100
43314/43314 [==============================] - 1s - loss: 0.6750 - val_los

0.65315415743121141

In [6]:
model.metrics_names

['loss']

In [7]:
model.predict(x_val)[0]

array([ 0.23099582,  0.06904652,  0.02361656,  0.00663695,  0.02984776,
        0.15226662,  0.06315273,  0.26238906,  0.16204797], dtype=float32)

In [8]:
y_val[0]

array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])